In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from utils import solr
from utils.logger import logger as log

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 100

In [130]:
df = pd.read_csv('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled.csv')

df = df.drop(['Unnamed: 0'], axis=1)

df = df.loc[df['zygosity'] == 'homozygote']

df = df.loc[(df['procedure_name'] == 'body weight') & (df['parameter_name'] == 'body weight')]

df = df.loc[~df['gene_symbol'].duplicated(keep=False)]

df = df[['gene_symbol', 'est_m_ea', 'se_m_ea', 'p_m_ea']]

df_effect_size = df.copy()

In [143]:
df = pd.read_pickle('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/gene_symbol_protein_sequence_all.pkl')

df = df[['gene_symbol', 'sequence']]

df = df_effect_size.merge(df, how='left', on='gene_symbol')

df = df.loc[df['sequence'].notna()]

df = df.groupby(['sequence'], group_keys=False, dropna=False).apply(lambda x: x.loc[x['est_m_ea'].abs() == x['est_m_ea'].abs().max()].iloc[0:1])

df = df.reset_index(drop=True)

df_effect_size_seq = df.copy()

In [ ]:
df = df_effect_size_seq.copy()

df

,gene_symbol,est_m_ea,se_m_ea,p_m_ea,sequence


In [42]:
def create_document(sequence, k):
    document = ' '.join([sequence[i:i+k] for i in range(0, len(sequence), k)])
    return document

df = df_effect_size_sequence.copy()

df['doc'] = df['sequence'].apply(lambda x: create_document(x, 2))

docs = df['doc'].tolist()

vectorizer = TfidfVectorizer(tokenizer=str.split, analyzer='word', ngram_range=(5,5), min_df=1)

X = vectorizer.fit_transform(docs)

y = df['est_m_ea'].to_numpy().copy()

X_train, X_test, y_train, y_test = train_test_split(X, y)

reg = LinearRegression().fit(X_train, y_train)

reg.score(X_test, y_test)

-1.2910579002777913

In [6]:
# import sys
# import pathlib
# from subprocess import run

# project_path = pathlib.Path('/data1/home/adpatter/mouse-aging-deep-phenotyping/adpatter').resolve()

# if project_path.as_posix() not in sys.path: sys.path.append(project_path.as_posix())
# from utils.logger import logger as log

# project_path = pathlib.Path('/data1/home/adpatter/mouse-aging-deep-phenotyping/adpatter')
# run_path = project_path.joinpath('run')
# main_path = project_path.joinpath('main')

In [7]:
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('extract_unidimensional_experiment_20220324_173145.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('extract_categorical_experiment_20220324_173145.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('combine_experiment.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('harmonize_experiment.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('vector_transform_experiment.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('vector_transform_experiment_wp1.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('vector_transform_experiment_wp2.ipynb')}", shell=True)

In [8]:
# # log.info('Run extract_unidimensional_experiment_20220324_173145.py')
# # run(f"python {run_path}/extract_unidimensional_experiment_20220324_173145.py', shell=True)

# # log.info('Run extract_categorical_experiment_20220324_173145.py')
# # run(f"python {run_path}/extract_categorical_experiment_20220324_173145.py', shell=True)

# log.info('Run combine_experiment.py')
# run(f"python {run_path.joinpath('combine_experiment.py')}", shell=True)

# log.info('Run harmonize_experiment.py')
# run(f"python {run_path.joinpath('harmonize_experiment.py')}", shell=True)

# log.info('Run vector_transform_experiment.py')
# run(f"python {run_path.joinpath('vector_transform_experiment.py')}", shell=True)

# log.info('Run vector_transform_experiment.py')
# run(f"python {run_path.joinpath('vector_transform_experiment_wp1.py')}", shell=True)

# log.info('Run vector_transform_experiment.py')
# run(f"python {run_path.joinpath('vector_transform_experiment_wp_2.py')}", shell=True)